In [ ]:
%load_ext autoreload
%autoreload 2

%reset

In [ ]:
# define training settings
NUM_EPOCHS = 100
BATCH_SIZE = 16

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from doren_bnn.mobilenet import MobileNet, NetType
from torchinfo import summary

NETTYPE = NetType.XNORPP_SCA
model = MobileNet(3, num_classes=10, nettype=NETTYPE).to(device)

summary(model, input_size=(BATCH_SIZE, 3, 32, 32))

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

criterion = CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=5e-6)
scheduler = CosineAnnealingWarmRestarts(optimizer, 25, eta_min=1e-4)

In [ ]:
# hyperparameters
ALPHA = 1e-1
LAMBDA = 1e-3

In [ ]:
from doren_bnn.utils import Dataset, Experiment

EXPERIMENT_ID = f"mobilenet-xnorpp-sca-0.2x-{ALPHA}-{LAMBDA}"
print(EXPERIMENT_ID)
experiment = Experiment(EXPERIMENT_ID, Dataset.CIFAR10, BATCH_SIZE)

In [ ]:
experiment.train(
    device,
    model,
    criterion,
    optimizer,
    scheduler,
    NUM_EPOCHS,
    resume=False,
    alpha=ALPHA,
    lamb=LAMBDA,
)

In [ ]:
experiment.test(device, model)